# Pynq-Z1 DES ACCELERATOR DEMO 
# HARDWARE API (HAPI)

* FULLY PARALLEL DES ACCELERATOR CODE
* Repository: https://github.com/IamVNIE/hapi-des
* Based-on: https://github.com/Xilinx/PYNQ

# Install package

In [ ]:
!sudo -H pip install --upgrade 'git+https://github.com/IamVNIE/hapi-des'

Step 1: Initialize DES ACCELERATOR -- RESET

In [1]:
from hapiDES import hapiDES
from pynq import pl
from pynq import Overlay
des=hapiDES()
des.reset_des_accel()

AttributeError: 'hapiDES' object has no attribute 'mmio'

Step 2: Initialize DES ACCELERATOR OPERATION -- SET TO ENCRYPTION

In [2]:
des.set_oper_encrypt()

DES ACCELERATOR IS SET TO ENCRYPT
-->SET A NEW KEY OR WILL USE THE PREVIOUS SET KEY


Step 3: Initialize ENCRYPTION KEY

In [3]:
MyKey=[0xfdcb0,0x167cde]
des.set_key(MyKey)

CRYPTO KEY UPDATED


Step 4: CHECK ENCRYPTION OPERATION ON SAMPLE DATA

In [4]:
input_data=[0xFE123410,0x12354567]
des.des_status()
des.encrypt(input_data)

DES ACCEL STATUS
ACTIVE (NOT IN RESET)
MODE:ENCRYPT


['0xdbd44175', '0x8cd6a970']

PERFORM OPERATION ON LARGE INPUT -- CAN INPUT A FILE

In [5]:
## This ia function to flatten the ASCII INPUT to HEX -- Has bug #Needs fixing
import time
import math
def data_formatter(txt):
    dataIn=''.join(hex(ord(x))[2:] for x in txt)
    #print(dataIn)
    #print(len(dataIn))
    vin=[]
    for i in range(0,len(dataIn),8):
        vin.append(''.join(map(str,dataIn[i:i+8])))
    vin=['0x' + s for s in vin]
    #print("INPUT")
    #print(vin)
    vin_bc=vin
    vin=[int(x,16) for x in vin]
    if len(vin)%2!=0:
        vin.append(0)
    return vin,vin_bc

# Encryption on a Message

In [6]:
txt ="Let us try to encrypt this"
data_2send,enc_in_hex=data_formatter(txt)
#print(enc_in_hex)
cryptRes=[]
j=0
TOTAL_BLOCKS=len(data_2send)
start_time = time.time()
for i in range(0,len(data_2send),2):
    cryptRes[j:j+1]=des.encrypt(data_2send[i:i+2])
    j=j+2
exec_time=time.time() - start_time
print("--- %s seconds ---" % (exec_time))
print("--- %s Blocks Per Second---" % ((math.pow(1, 1))*(TOTAL_BLOCKS/2)/exec_time))
print("ENCRYPTED CIPHER TEXT")
print(cryptRes)

--- 0.0022466182708740234 seconds ---
--- 1780.45378329619 Blocks Per Second---
ENCRYPTED CIPHER TEXT
['0x176c295', '0xd1470f4a', '0x6e8c69a9', '0xc3023084', '0x845b63ce', '0x9ec95823', '0xdedbc128', '0xb0042c0b']


# Decryption of encrypted message

In [7]:
des.set_oper_decrypt()
decipher=[]
j=0
TOTAL_BLOCKS=len(cryptRes)
cryptRes=[int(x,16) for x in cryptRes]
#print(cryptRes)
start_time = time.time()
for i in range(0,len(cryptRes),2):
    decipher[j:j+1]=des.decrypt(cryptRes[i:i+2])
    j=j+2
exec_time=time.time() - start_time
print("--- %s seconds ---" % (exec_time))
print("--- %s Blocks Per Second---" % ((math.pow(1, 1))*(TOTAL_BLOCKS/2)/exec_time))
print(decipher)

DES ACCELERATOR IS SET TO DECRYPT
-->SET A NEW KEY OR WILL USE THE PREVIOUS SET KEY
--- 0.0028123855590820312 seconds ---
--- 1422.28009494744 Blocks Per Second---
['0x4c657420', '0x75732074', '0x72792074', '0x6f20656e', '0x63727970', '0x74207468', '0x6973', '0x0']


Verifying if we got the original input

In [8]:
#bool(set(decipher) & set(enc_in_hex))
print(set(decipher)^set(enc_in_hex))
#print(x) for x,y in (decipher,enc_in_hex) if x!=y

{'0x0'}


# Print the message back in ascii

In [9]:
dec=(''.join(map(str,decipher)))
dec=dec.replace("0x","")
print(dec)
ascii_string =''
x = 0
y = 2
l = len(dec)
while y <= l:
    ascii_string += chr(int(dec[x:y], 16))
    x += 2
    y += 2
print (ascii_string)

4c65742075732074727920746f20656e637279707420746869730
Let us try to encrypt this
